In [2]:
import numpy as np
from obj_func import obj_func
from read_instance import read_instance
import random
import seaborn as sns
from random import randint
import sys 

In [3]:
file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\a280_n279_bounded-strongly-corr_01.ttp","r")
#file=open(r"E:\monit\Documents\TTP\Instancias\a280-ttp\my_test.ttp","r")
dic=read_instance(file)


In [4]:
def create_knap_pop(dic,c):
    pop_knap=np.zeros((c,dic["number of items"]))
    for i in range(pop_knap.shape[0]):
        item_pick=[]
        for x in range(dic["number of items"]): 
            item_pick.append(random.randint(0, 1))
        item_pick2=np.array(item_pick)
        pop_knap[i]=item_pick2
    return pop_knap

In [5]:
def create_distance_matrix(dic):
    chords=dic["coords citites"]
    matrix=np.zeros((len(chords),len(chords)))
    ind1=0
    for x in chords:
        ind2=0
        for f in chords:
            point1=np.array((int(x[0]),int(x[1])))
            point2=np.array((int(f[0]),int(f[1])))
            matrix[ind1,ind2]=np.linalg.norm(point1-point2)
            ind2=ind2+1
        ind1=ind1+1
    return matrix

In [6]:
def create_tsp_pop(dic,x,matrix):
    pop_tsp=np.zeros((x,dic["number cities"]-1))
    minimo=np.amin(matrix[matrix>0])
    maximo=np.amax(matrix[matrix>0])
    #print(minimo,maximo,pop_tsp.shape[1])
    for i in range(pop_tsp.shape[0]):
        #order_cities=list(range(2,dic["number cities"]+1 ))
        dist=(maximo-minimo) * np.random.random_sample((pop_tsp.shape[1] )) +minimo    
        pop_tsp[i,:]=dist
    return pop_tsp

In [7]:
def cont_to_disc_tsp(dic,pop_tsp,matrix):
    pop_tsp_disc=np.zeros((pop_tsp.shape))
    matrix[:,0]=0
 
    for x in range(pop_tsp.shape[0]):
        line_check=matrix[0,:].copy()
        orden=[]
        for i in pop_tsp[x,:]:
            #print(i,"i")
            diff=abs(line_check-i)
            pick_list=list(zip(*np.where(diff==np.amin(diff))))
            pick= int(random.choice(pick_list)[0])
            #pick=int(pick_list[0][0])
            orden.append(pick+1)
            line_check=matrix[pick,:].copy()
            for l in orden:
                line_check[l-1]=0
            #print(diff,"diff")
            #print(pick,'city')
            #print(line_check,"newline'")
        #print(orden,"orden_ciudades",len(set(orden)),"unicos",len(orden),'largo del orden')
        pop_tsp_disc[x,:]=orden
    return pop_tsp_disc


In [8]:
def fit_loop(dic,pop_tsp,pop_knap):
    fitness=np.zeros((pop_tsp.shape[0]))
    for l in range(pop_tsp.shape[0]):
        fitness[l]=obj_func(dic,pop_tsp[l,:],pop_knap[l,:])
    return fitness

In [9]:

def diff_evolution(fitness,pop_tsp,pop_knap,f,r,matrix,dic):
    dim=np.shape(pop_tsp)[0]
    minimo=np.amin(matrix[matrix>0])
    maximo=np.amax(matrix[matrix>0])
    new_population_tsp=np.zeros(np.shape(pop_tsp))
    new_population_knap=np.zeros(np.shape(pop_knap))
    for k in range(dim): 
        if random.uniform(0,1)>r :
            selector=random.sample(range(dim), 3)
            #print(selector)
            #diff evolution for tsp
            pre_pop_tsp=pop_tsp[2].copy()  + (f*(pop_tsp[selector[0]].copy()-pop_tsp[selector[1]].copy()))
            pre_pop_tsp=np.reshape(np.clip(pre_pop_tsp,minimo,maximo),(1,pre_pop_tsp.shape[0]))
            pre_pop_tsp_fit=(cont_to_disc_tsp(dic,pre_pop_tsp,matrix)).astype(np.int32)
            #print(pre_pop_tsp_fit,cont_to_disc_tsp(dic,np.reshape(pop_tsp[k,:],(1,pop_tsp.shape[1])),matrix))
           
            #diff evolution for knap, bin dif evolutio
            pre_nap=pop_knap[selector[0],:].copy()
            #print(pop_knap[selector[0],:],"before")
            #print(pop_knap[selector[1],:],pop_knap[selector[2],:],"bins")
            for l in range(pop_knap.shape[1]):
                if pop_knap[selector[1],l]!=pop_knap[selector[2],l] and random.uniform(0,1)>f:
                    pre_nap[l]=1-pre_nap[l]
            #print(pre_nap,'after')
            fitness1=fitness[k]
            fitness2=obj_func(dic,pre_pop_tsp_fit[0,:],pre_nap)            
            if fitness2>fitness1:
                new_population_tsp[k,:]=np.reshape(pre_pop_tsp,(pre_pop_tsp.shape[1]))
                new_population_knap[k,:]=pre_nap
            else:
                new_population_tsp[k,:]=pop_tsp[k,:]
                new_population_knap[k,:]=pop_knap[k,:]
        else:
            new_population_tsp[k,:]=pop_tsp[k,:]
            new_population_knap[k,:]=pop_knap[k,:]
        #print(fitness1,fitness2)
    
     
    return new_population_tsp,new_population_knap
    
    

In [ ]:
ps=200#pop length
#iterations=1000
cr=0.9
f=0.8
matrix=create_distance_matrix(dic)
pop_tsp=create_tsp_pop(dic,ps,matrix)
pop_knap=create_knap_pop(dic,ps)
pop_tsp_for_fit=cont_to_disc_tsp(dic,pop_tsp,matrix) 
fitness=fit_loop(dic,pop_tsp_for_fit,pop_knap)
z=True
l=0
lastfit=np.amax(fitness)
count=0
while z :
    pop_tsp,pop_knap=diff_evolution(fitness,pop_tsp,pop_knap,f,cr,matrix,dic)
    pop_tsp_for_fit=cont_to_disc_tsp(dic,pop_tsp,matrix)
    fitness=fit_loop(dic,pop_tsp_for_fit,pop_knap)
    print("generation: ",l, " maximo: ", np.amax(fitness))  
    l=l+1
    
    if lastfit>=np.amax(fitness):
        count=count+1
    else:
        count=0
    lastfit=np.amax(fitness)
    if count==100:
        z=False
    
p1 = sns.heatmap(np.reshape(fitness,(1,fitness.shape[0])))    

generation:  0  maximo:  -1445500.913127531
generation:  1  maximo:  -1485198.2195056733
generation:  2  maximo:  -1416579.6664955523
generation:  3  maximo:  -1449569.953982819
generation:  4  maximo:  -1446265.446814352
generation:  5  maximo:  -1416428.0107658138
generation:  6  maximo:  -1425667.456907431
generation:  7  maximo:  -1448020.6130787495
generation:  8  maximo:  -1389305.7621756983
generation:  9  maximo:  -1412784.6675254183
generation:  10  maximo:  -1417610.577292562
generation:  11  maximo:  -1342293.7884928763
generation:  12  maximo:  -1401573.0380248968
generation:  13  maximo:  -1363991.0301466128
generation:  14  maximo:  -1333796.5623947156
generation:  15  maximo:  -1434042.0600062814
generation:  16  maximo:  -1386190.7667434418
generation:  17  maximo:  -1390731.0782458638
generation:  18  maximo:  -1294126.8113337832
generation:  19  maximo:  -1361589.7544798376
generation:  20  maximo:  -1366395.7891353387
generation:  21  maximo:  -1346787.5110626959
gen

generation:  178  maximo:  -1108226.9047796517
generation:  179  maximo:  -1101410.258608449
generation:  180  maximo:  -1118217.3400207341
generation:  181  maximo:  -1132050.4833604472
generation:  182  maximo:  -1076340.336118178
generation:  183  maximo:  -1089446.3809998175
generation:  184  maximo:  -1133127.0723247777
generation:  185  maximo:  -1147093.5777499627
generation:  186  maximo:  -1116721.1876497527
generation:  187  maximo:  -1121778.6247975812
generation:  188  maximo:  -1093441.9334837901
generation:  189  maximo:  -1145519.1746855923
generation:  190  maximo:  -1127157.335878487
generation:  191  maximo:  -1130970.5355361465
generation:  192  maximo:  -1153836.5557283664
generation:  193  maximo:  -1134582.7726473836
generation:  194  maximo:  -1108197.261565195
generation:  195  maximo:  -1119516.3907030416
generation:  196  maximo:  -1121753.3398423109
generation:  197  maximo:  -1148882.267342126
generation:  198  maximo:  -1143376.5550561394
generation:  199  

generation:  355  maximo:  -941690.100765773
generation:  356  maximo:  -949787.6692037137
generation:  357  maximo:  -928144.7991827875
generation:  358  maximo:  -982106.9299333297
generation:  359  maximo:  -952435.597935192
generation:  360  maximo:  -962954.7511525067
generation:  361  maximo:  -930294.353471696
generation:  362  maximo:  -925054.9879303062
generation:  363  maximo:  -968632.79883555
generation:  364  maximo:  -954636.5935592253
generation:  365  maximo:  -914274.129178203
generation:  366  maximo:  -922236.4160612158
generation:  367  maximo:  -935872.8999970288
generation:  368  maximo:  -958712.7748583889
generation:  369  maximo:  -877264.7196497568
generation:  370  maximo:  -951812.3388594505
generation:  371  maximo:  -945330.111138895
generation:  372  maximo:  -939757.7092618415
generation:  373  maximo:  -879085.1418373997
generation:  374  maximo:  -949759.1694311528
generation:  375  maximo:  -925413.0431436647
generation:  376  maximo:  -980530.225863

generation:  534  maximo:  -834257.8717989814
generation:  535  maximo:  -863509.3355794369
generation:  536  maximo:  -864064.0866101258
generation:  537  maximo:  -860346.7730163792
generation:  538  maximo:  -805399.6069057487
generation:  539  maximo:  -855069.8882943431
generation:  540  maximo:  -812603.0262147184
generation:  541  maximo:  -850465.3751423309
generation:  542  maximo:  -838886.9337590059
generation:  543  maximo:  -874857.8866307972
generation:  544  maximo:  -811620.5202626899
generation:  545  maximo:  -873749.7201410579
generation:  546  maximo:  -800950.4610498195
generation:  547  maximo:  -810030.3251283402
generation:  548  maximo:  -797261.8464962629
generation:  549  maximo:  -843215.7139936608
generation:  550  maximo:  -861377.0822780696
generation:  551  maximo:  -787821.5056085401
generation:  552  maximo:  -842314.528367078
generation:  553  maximo:  -837961.5339717352
generation:  554  maximo:  -845260.8977950307
generation:  555  maximo:  -851301.

generation:  713  maximo:  -677266.3687510106
generation:  714  maximo:  -681389.1797536657
generation:  715  maximo:  -683800.3389587009
generation:  716  maximo:  -664170.4794336227
generation:  717  maximo:  -684643.1137700093
generation:  718  maximo:  -678302.8690713105
generation:  719  maximo:  -682835.3626108334
generation:  720  maximo:  -659191.594308874
generation:  721  maximo:  -683093.8143313306
generation:  722  maximo:  -692706.7006315948
generation:  723  maximo:  -693989.583584984
generation:  724  maximo:  -710273.2695334769
generation:  725  maximo:  -666431.2590691088
generation:  726  maximo:  -665876.9014026165
generation:  727  maximo:  -664791.4707769965
generation:  728  maximo:  -676180.2943929589
generation:  729  maximo:  -660707.38214235
generation:  730  maximo:  -644052.668596801
generation:  731  maximo:  -649025.7897258392
generation:  732  maximo:  -681414.5265171224
generation:  733  maximo:  -705447.8387287547
generation:  734  maximo:  -657151.0730

...

generation:  10530  maximo:  -151463.50677309962
generation:  10531  maximo:  -150388.0285258969
generation:  10532  maximo:  -151465.30379656213
generation:  10533  maximo:  -152386.14462152185
generation:  10534  maximo:  -147971.82382471318
generation:  10535  maximo:  -154211.75203560933
generation:  10536  maximo:  -153047.20416020622
generation:  10537  maximo:  -152106.17152235372
generation:  10538  maximo:  -152562.85819272956
generation:  10539  maximo:  -149679.0470454789
generation:  10540  maximo:  -148607.88190873817
generation:  10541  maximo:  -149659.9349781149
generation:  10542  maximo:  -150296.95865757292
generation:  10543  maximo:  -152137.00496654166
generation:  10544  maximo:  -149925.2439954306
generation:  10545  maximo:  -152892.7298691495
generation:  10546  maximo:  -153935.67327306778
generation:  10547  maximo:  -154683.66863065612
generation:  10548  maximo:  -150667.51582149178
generation:  10549  maximo:  -149201.609940398
generation:  10550  maximo:

generation:  10699  maximo:  -151717.9516328012
generation:  10700  maximo:  -148856.63621132748
generation:  10701  maximo:  -148888.47027587908
generation:  10702  maximo:  -150287.8123350141
generation:  10703  maximo:  -150750.09690310303
generation:  10704  maximo:  -152460.10683032946
generation:  10705  maximo:  -152778.6020658794
generation:  10706  maximo:  -152362.69947339108
generation:  10707  maximo:  -151473.28071518522
generation:  10708  maximo:  -147608.36786130178
generation:  10709  maximo:  -149364.38711517968
generation:  10710  maximo:  -149751.9810099177
generation:  10711  maximo:  -149868.4373273806
generation:  10712  maximo:  -152087.62466883557
generation:  10713  maximo:  -153340.64640871348
generation:  10714  maximo:  -147489.3082320659
generation:  10715  maximo:  -153127.20759765434
generation:  10716  maximo:  -148458.11389746392
generation:  10717  maximo:  -142614.57899330222
generation:  10718  maximo:  -148897.63824821066
generation:  10719  maximo

generation:  10868  maximo:  -150777.44625831363
generation:  10869  maximo:  -153710.2716168302
generation:  10870  maximo:  -151963.81933524885
generation:  10871  maximo:  -147155.46405301444
generation:  10872  maximo:  -150572.95923700772
generation:  10873  maximo:  -150627.70677536895
generation:  10874  maximo:  -150304.1747120242
generation:  10875  maximo:  -151715.31629091298
generation:  10876  maximo:  -152598.95826783884
generation:  10877  maximo:  -153720.6850977287
generation:  10878  maximo:  -149264.14730424606
generation:  10879  maximo:  -152618.04103587355
generation:  10880  maximo:  -150775.55953678765
generation:  10881  maximo:  -151611.7927840528
generation:  10882  maximo:  -147084.30992383178
generation:  10883  maximo:  -153122.44226542188
generation:  10884  maximo:  -152332.89496144676
generation:  10885  maximo:  -150495.69642482212
generation:  10886  maximo:  -150510.7397208247
generation:  10887  maximo:  -148605.78290097942
generation:  10888  maxim

generation:  11037  maximo:  -151921.58693151115
generation:  11038  maximo:  -152353.76163906947
generation:  11039  maximo:  -150773.33235934234
generation:  11040  maximo:  -149572.38454846118
generation:  11041  maximo:  -153362.8141643437
generation:  11042  maximo:  -151941.33209894234
generation:  11043  maximo:  -151088.0000498802
generation:  11044  maximo:  -148768.61471267542
generation:  11045  maximo:  -148615.71360987594
generation:  11046  maximo:  -151253.4665384397
generation:  11047  maximo:  -153007.96600445316
generation:  11048  maximo:  -151316.40563154678
generation:  11049  maximo:  -150190.74104717237
generation:  11050  maximo:  -145750.49928545736
generation:  11051  maximo:  -146120.69610732427
generation:  11052  maximo:  -149815.60581088386
generation:  11053  maximo:  -152967.72505373755
generation:  11054  maximo:  -150781.799976008
generation:  11055  maximo:  -150300.3799074798
generation:  11056  maximo:  -148070.2477854703
generation:  11057  maximo:

generation:  11206  maximo:  -149205.63381403012
generation:  11207  maximo:  -152152.0183890125
generation:  11208  maximo:  -149322.6652434347
generation:  11209  maximo:  -154398.02003203498
generation:  11210  maximo:  -150898.48079096875
generation:  11211  maximo:  -152014.3790937688
generation:  11212  maximo:  -149357.6081699494
generation:  11213  maximo:  -149215.48430533474
generation:  11214  maximo:  -154064.29965729563
generation:  11215  maximo:  -153848.02665982101
generation:  11216  maximo:  -149018.54900982385
generation:  11217  maximo:  -150740.04404041803
generation:  11218  maximo:  -151692.6719622672
generation:  11219  maximo:  -151145.35771792225
generation:  11220  maximo:  -152255.74678338188
generation:  11221  maximo:  -147151.67674339906
generation:  11222  maximo:  -153761.10940302568
generation:  11223  maximo:  -149050.13304227171
generation:  11224  maximo:  -154229.5228867133
generation:  11225  maximo:  -149360.30308342798
generation:  11226  maximo